###**Modelo con preprocesado B y LinearRegression**

####**Cargar librerías**

In [153]:
import pandas as pd
import numpy as np
import os
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

####**Carga del conjunto de datos Train y Test**

In [154]:
os.environ['KAGGLE_CONFIG_DIR'] = "."  #Se establece la variable de entorno al directorio actual

In [155]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia  # Se descargan los archivos de la competencia directamente desde la API de Kaggle

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [156]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip # Se descomprime el fichero para acceder al contenido

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [157]:
# Cargar train y test
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

##**Limpieza de datos**

#####**Eliminación de las columnas innecesarias**

En esta versión del preprocesado eliminamos otras columnas del dataset que no aportan información útil para el modelo. Esto incluye columnas duplicadas y columnas redundantes con otras variables.

In [158]:
del(train["ID"])

del(train["F_TIENEINTERNET.1"]) # Columna duplicada
del(test["F_TIENEINTERNET.1"])

del(train["F_TIENELAVADORA"]) # Redundate con estrato socioeconómico
del(test["F_TIENELAVADORA"])

del(train["F_TIENEAUTOMOVIL"]) # Redundate con estrato socioeconómico
del(test["F_TIENEAUTOMOVIL"])

del(train["E_PAGOMATRICULAPROPIO"]) # Redundate con valor de matrícula
del(test["E_PAGOMATRICULAPROPIO"])

del(train["PERIODO_ACADEMICO"]) # No aporta información relevante
del(test["PERIODO_ACADEMICO"])

del(train["E_PRIVADO_LIBERTAD"]) # No aporta información relevante
del(test["E_PRIVADO_LIBERTAD"])

####**Tratamiento de Datos Faltantes**


In [159]:
col_Nanh = train.isna().sum()
col_Nanh[col_Nanh!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENECOMPUTADOR,38103
F_EDUCACIONMADRE,23664


Imputamos los valores faltantes de ciertas columnas categóricas usando la moda.

In [160]:
# Columnas a imputar con la moda
cols_a_imputar = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE',
                  'F_TIENEINTERNET', 'F_TIENECOMPUTADOR']

# Calcular la moda en TRAIN
moda_train = train[cols_a_imputar].mode().iloc[0]

# Imputación en TRAIN usando su propia moda
train[cols_a_imputar] = train[cols_a_imputar].fillna(moda_train)

# Imputación en TEST usando la moda del TRAIN para no usar los datos de test
test[cols_a_imputar] = test[cols_a_imputar].fillna(moda_train)

# Verificación
print("Nulos en train:\n", train[cols_a_imputar].isnull().sum())
print("Nulos en test:\n", test[cols_a_imputar].isnull().sum())

Nulos en train:
 F_ESTRATOVIVIENDA    0
F_EDUCACIONPADRE     0
F_EDUCACIONMADRE     0
F_TIENEINTERNET      0
F_TIENECOMPUTADOR    0
dtype: int64
Nulos en test:
 F_ESTRATOVIVIENDA    0
F_EDUCACIONPADRE     0
F_EDUCACIONMADRE     0
F_TIENEINTERNET      0
F_TIENECOMPUTADOR    0
dtype: int64


A continuación, se rellenan los valores faltantes por "missing" en los valores nulos faltantes.

In [161]:
df_train = train.fillna("missing")
df_test = test.fillna("missing")

#####**Codificación de Variables Categóricas con One Hot**

Para esta versión solo ciertas columnas son transformadas mediante One Hot Encodin (programa académico y el departamento).

In [162]:
from scipy import sparse as sp

#Estas funciones permiten trasnformar las variables categóricas en formato One-Hot
def to_onehot(x): # Convierte una variable categórica a formato One Hot
    values = np.unique(x)
    r = np.r_[[np.argwhere(i == values)[0][0] for i in x]]
    onehot_sparse = sparse.csr_matrix(
        (np.ones(len(r)), (np.arange(len(r)), r)),
        shape=(len(r), len(values))
    )
    return onehot_sparse, values

# Reemplaza una columna por su versión One Hot
def replace_column_with_onehot(d, col):
    assert sum(d[col].isna()) == 0, "column must have no NaN values"
    k_sparse, values = to_onehot(d[col].values)
    k = pd.DataFrame.sparse.from_spmatrix(
        k_sparse,
        columns=["%s_%s" % (col, values[i]) for i in range(k_sparse.shape[1])],
        index=d.index
    )
    r = k.join(d)
    del (r[col])
    return r

# Aplica One-Hot Encoding a múltiples columnas categóricas
def aplicar_onehot_a_varias(df, columnas):
    df_datos = df.copy()
    for col in columnas:
        df_datos = replace_column_with_onehot(df_datos, col)
    return df_datos


In [163]:
# Columnas para aplicar one-hot
cols_onehot = [
    "E_PRGM_DEPARTAMENTO",
    "E_PRGM_ACADEMICO"
]

# Aplicamos la función a train y test
df_train_onehot = aplicar_onehot_a_varias(df_train, cols_onehot)
df_test_onehot = aplicar_onehot_a_varias(df_test, cols_onehot)

# Verificamos shapes
print(df_train_onehot.shape, df_test_onehot.shape)

(692500, 991) (296786, 962)


####**Tratamiento de Variables Categóricas Ordinales**

Para el tratamiento de variables categóricas ordinales se decide variar levemente el mapeo, por ejemplo en el caso del rendimiento global, que ahora inicia desde 1 en lugar de 0.

In [164]:
def codificar_ordinal(df, mapeos):
    df_copia = df.copy()
    for col, mapa in mapeos.items():
        if col in df_copia.columns:
            df_copia[col] = df_copia[col].map(mapa)
    return df_copia

mapeos_train = {
    'E_VALORMATRICULAUNIVERSIDAD': {
        'missing': -1,
        'No pagó matrícula': 0,
        'Menos de 500 mil': 0.25,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.25,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Más de 7 millones': 8
    },
    'E_HORASSEMANATRABAJA': {
        'missing': -1,
        '0': 0,
        'Menos de 10 horas': 5,
        'Entre 11 y 20 horas': 15,
        'Entre 21 y 30 horas': 25,
        'Más de 30 horas': 35
    },
    'F_ESTRATOVIVIENDA': {
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    },
    'F_EDUCACIONPADRE': {
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10
    },
    'F_EDUCACIONMADRE': {
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10
    },
    "F_TIENEINTERNET": {
        "No": 0,
        "Si": 1
    },
    "F_TIENECOMPUTADOR": {
        "No": 0,
        "Si": 1
    },
    'RENDIMIENTO_GLOBAL': {  # Intentamos con mapeo diferente
        'bajo': 1,
        'medio-bajo': 2,
        'medio-alto': 3,
        'alto': 4
    }
}

mapeos_test = {
    'E_VALORMATRICULAUNIVERSIDAD': {
        'missing': -1,
        'No pagó matrícula': 0,
        'Menos de 500 mil': 0.25,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.25,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Más de 7 millones': 8
    },
    'E_HORASSEMANATRABAJA': {
        'missing': -1,
        '0': 0,
        'Menos de 10 horas': 5,
        'Entre 11 y 20 horas': 15,
        'Entre 21 y 30 horas': 25,
        'Más de 30 horas': 35
    },
    'F_ESTRATOVIVIENDA': {
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    },
    'F_EDUCACIONPADRE': {
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10
    },
    'F_EDUCACIONMADRE': {
        'No sabe': -1,
        'No Aplica': 0,
        'Ninguno': 1,
        'Primaria incompleta': 2,
        'Primaria completa': 3,
        'Secundaria (Bachillerato) incompleta': 4,
        'Secundaria (Bachillerato) completa': 5,
        'Técnica o tecnológica incompleta': 6,
        'Técnica o tecnológica completa': 7,
        'Educación profesional incompleta': 8,
        'Educación profesional completa': 9,
        'Postgrado': 10
    },
    "F_TIENEINTERNET": {
        "No": 0,
        "Si": 1
    },
    "F_TIENECOMPUTADOR": {
        "No": 0,
        "Si": 1
    }
}

df_train_ord = codificar_ordinal(df_train_onehot, mapeos_train)
df_test_ord  = codificar_ordinal(df_test_onehot, mapeos_test)

####**Normalización de Columnas**

En esta versión se deciden normalizar todas las variables numéricas para asegurar que estén en la misma escala.

In [167]:
from sklearn.preprocessing import StandardScaler

escalar = StandardScaler()
# Columnas que realmente son numéricas y deben normalizarse
columnas_a_normalizar = [
    "E_VALORMATRICULAUNIVERSIDAD",
    "E_HORASSEMANATRABAJA",
    "INDICADOR_1",
    "INDICADOR_2",
    "INDICADOR_3",
    "INDICADOR_4"
]


escalar = StandardScaler()
# Ajustar el escalador a los datos y transformar las columnas
df_train_ord[columnas_a_normalizar] = escalar.fit_transform(df_train_ord[columnas_a_normalizar])
df_test_ord[columnas_a_normalizar]  = escalar.transform(df_test_ord[columnas_a_normalizar])

df_train_final = df_train_ord.copy()
df_test_final = df_test_ord.copy()

###**Modelo LinearRegression**

Se aplica un modelo de regresión lineal y se evalúa su desempeño utilizando una partición de los datos de train en 70% para entrenamiento y 30% para prueba.

In [169]:
# Variable objetivo
y = df_train_final["RENDIMIENTO_GLOBAL"]

# Variables predictoras
X = df_train_final.drop(columns=["RENDIMIENTO_GLOBAL"])

# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Ajustar modelo
estimator = LinearRegression()
estimator.fit(X_train, y_train)

# Predicciones
y_pred = estimator.predict(X_test)

# Convertir a clases discretas
y_pred_clases = np.clip(np.round(y_pred), 0, 3).astype(int)

# Calcular accuracy
acc = accuracy_score(y_test, y_pred_clases)
print("Accuracy aproximado:", acc)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Accuracy aproximado: 0.3016317689530686


###**Técnicas de Validación**

Se aplica un modelo de regresión lineal y se evalúa su desempeño usando como métrica el error absoluto medio relativo (MAE).

In [170]:
# Construimos nuestra medida de desempeño
def rel_mae_mean(estimator, X, y):
    preds = estimator.predict(X)
    mae = np.mean(np.abs(preds - y))
    return mae / np.mean(y)

def bootstrap_score(estimator, X, y, test_size):
    trscores, tsscores = [], []
    for _ in range(2):
        Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=test_size)
        estimator.fit(Xtr, ytr)
        trscores.append(rel_mae_mean(estimator, Xtr, ytr))
        tsscores.append(rel_mae_mean(estimator, Xts, yts))
    return (np.mean(trscores), np.std(trscores)), (np.mean(tsscores), np.std(tsscores))

estimator_bt = LinearRegression()
(trmean, trstd), (tsmean, tsstd) = bootstrap_score(estimator_bt, X, y, test_size=0.3)

print("train score %.3f (±%.4f)" % (trmean, trstd))
print("test score  %.3f (±%.4f)" % (tsmean, tsstd))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklea

train score 0.325 (±0.0001)
test score  0.326 (±0.0003)


####**Validación Cruzada**

#####**Uso de ShuffleSplit**

Se utilizó validación cruzada con ShuffleSplit. Se realizaron tres particiones del dataset.



In [171]:
# Modelo
estimator_sh = LinearRegression()

# ShuffleSplit CV
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=7)

# Evaluación con tu métrica personalizada
z = cross_val_score(estimator_sh, X, y, cv=cv, scoring=rel_mae_mean)
print(z)
print("test score %.3f (±%.4f)" % (np.mean(z), np.std(z)))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklea

[0.32534853 0.32682879 0.32606809]
test score 0.326 (±0.0006)


####**Generación del CSV - Kaggle**

In [172]:
# Alinear columnas de test con las de train
# Se eliminan atributos adicionales presentes solo en test (programas académicos sobre los cuales train no tiene información)
X_test_aligned = df_test_final.reindex(columns=X.columns, fill_value=0)

# Generar predicciones con el modelo entrenado
y_pred = estimator.predict(X_test_aligned)

# Convertir predicciones continuas a clases discretas
y_pred_clases = np.clip(np.round(y_pred), 0, 3).astype(int)

# Mapear números a originales
num_to_label = {0:'bajo', 1:'medio-bajo', 2:'medio-alto', 3:'alto'}
y_pred_labels = [num_to_label[i] for i in y_pred_clases]

# Guardar resultados en CSV
resultado = pd.DataFrame({
    "ID": df_test_final["ID"],
    "RENDIMIENTO_GLOBAL": y_pred_labels
})

resultado.to_csv("predicciones_test.csv", index=False)
print("Predicciones generadas")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Predicciones generadas
